**To use the examples in this chapter, first run the code below to import the right libraries.**

In [ ]:
# =================================
# Imports
# =================================
from PyCh import *
from numpy import random
from dataclasses import dataclass
import math

# 8 Channels
In the previous chapter processes have been introduced. This chapter describes channels.
A channel connects two processes and is used for the transfer of data or just signals.
One process is the sending process, the other process is the receiving process.
Communication between the processes takes place instantly when both processes are willing to communicate,
 this is called *synchronous* communication.

## 8.1 A channel
So now that you know what a channel is, you need to know how to use them in your models. You can add a channel to environment `env` by instantiating `Channel(env)`. A channel always has the simulation environment in which it operates as its single argument.

To communicate across a channel we use *communication events*. We can send an entity over a channel by using the communication event `Channel.send(entity)`,  and we can receive over the same channel by using the communication event `Channel.receive()`. Unlike timeout events or processes, communication events do not start as soon as they are defined. Instead, `Environment.execute(communication event)` is used to start the communication event. A channel can transmit any object as entity; for example an integer, a string, or a custom data type. 

Communication over a channel requires a process which is sending, and a process which is receiving. If a process has no other process to communicate with, then it will have to wait. Communication is only completed once the receiving process has received the entity from the sending process. If a process uses the `yield` statement, such as with `yield Environment.execute(communication event)`, then the process waits till communication has completed (from both sides!) before it continues.



### 8.1.1 Producer and consumer example

The figure below shows the two processes `P` and `C`, connected by channel `a`. Processes are denoted by circles, and channels are denoted by directed arrows in the figure. The arrow denotes the direction of communication. Process `P` is the sender or producer, process `C` is the receiver or consumer.

| Figure 8.1: A producer and a consumer |
- 
<img src="figures/8-1.png" width=75%>
<a id='fig:8-1'></a>

In the model below, the producer `P` sends a stream of five integer values to consumer `C`. 
The producer sends integers `i` over channel `a`, which is realized through `env.execute(a.send(i))`. It sends the sequence of values `0, 1, 2, 3, 4` one after the other. The consumer receives these values one by one over channel `a` with `i = yield env.execute(a.receive())`. It then prints these values as output.

The simulation runs until there are no more events to execute. So in the example below the simulation will end once the producer has sent all five integers, even though the process of the consumer is supposed to go on forever (on account of its `while True:` loop). This is because none of the processes are active; all processes are either finished (the producer), or are waiting for other processes to do something (the consumer).

In [ ]:
@process
def producer(env, c_out):
    for i in range(5):
        yield env.execute(c_out.send(i))

@process
def consumer(env, c_in):
    while True:
        i = yield env.execute(c_in.receive())
        print(i)

def model():
    env = Environment()
    a = Channel(env)
    P = producer(env, a)
    C = consumer(env, a)

    env.run()
    print("The simulation has finished.")
    
model()

--- 
### 8.1.2 Synchronization signals
Earlier we said that we can transmit any type of entity over a channel. What we did not yet tell, is that it is also possible to send an empty signal by using `Channel.send(None)`, these signals are called *synchronization signals*, as they just synchronize actions between different processes without transmitting data.

Below we see an example in which synchronization signals are used; the producer does not send any data (it sends `None`). We first define our communication events through `sending = a.send(None)` and `receiving = a.receive()`, before executing and yielding them. This also allows us to later access the received entity through `receiving.entity` (which in this example is `None`).

In [ ]:
@process
def producer(env, c_out):
    for i in range(5):
        sending = c_out.send(None)
        yield env.execute(sending)

@process
def consumer(env, c_in):
    while True:
        receiving = c_in.receive()
        yield env.execute(receiving)
        print(receiving.entity)

def model():
    env = Environment()
    a = Channel(env)
    P = producer(env, a)
    C = consumer(env, a)

    env.run()
    
model()

## 8.2 Two channels
A process can have more than one channel, allowing interaction with several other processes.

The next example shows two channels, `a` and `b`, and three processes, generator `G`, server `S` and exit `E`. Process `G` is connected via channel `a` to process `S` and process `S` is connected via channel `b` to process `E`.
The model is given in the figure below. 


| Figure 8.2: A generator, a server and an exit |
- 
<img src="figures/8-2.png" width=75%>
<a id='fig:8-2'></a>

The model below shows an example of such a configuration. Process `G` sends a stream of integer values `0, 1, 2, 3, 4` to another process via channel `a`. 
Process `S` receives a value via channel `a`, assigns this value to variable `x`, doubles the value of the variable, and sends the value of the variable via `b` to another process.
Process `E` receives a value via channel `b`, assigns this value to the variable `x`, and prints this value.

After printing these five lines, process `G` stops, and processes `S` and `E` are no longer able to receive anything, so the simulation ends.


In [ ]:
@process
def Generator(env, c_out):
    for x in range(5):
        sending = c_out.send(x)
        yield env.execute(sending)

@process
def Server(env, c_in, c_out):
    while True:
        receiving = c_in.receive()
        x = yield env.execute(receiving)
        
        sending = c_out.send(2*x)
        yield env.execute(sending)

@process
def Exit(env, c_in):
    while True:
        receiving = c_in.receive()
        x = yield env.execute(receiving)
        print(f"The Exit process received {x}")

def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    
    G = Generator(env, a)
    S = Server(env, a, b)
    E = Exit(env, b)
    
    env.run()
    
model()

## 8.3 More senders or receivers
Channels send a message (or a signal in case of synchronization channels) from one sender to one receiver.
It is however allowed to give the same channel to several sender or receiver processes. The channel chooses a sender and a receiver before each communication.

The following example gives an illustration, see the figure below.

| Figure 8.3: A generator, two servers and an exit |
- 
<img src="figures/8-3.png" width=75%>
<a id='fig:8-3'></a>

Suppose that only `G` and `S0` want to communicate. The channel wil choose a sender (namely `G`) and a receiver (process `S0`), and let both processes communicate with each other.
When sender `G`, and both receivers (`S0` and `S1`), want to communicate,
the channel chooses a sender (`G` as it is the only sender available to the channel), and a receiver (either process `S0` or process `S1`), and it lets the chosen processes communicate with each other. This selection process is random.

Sharing a channel in this way allows to send data to receiving processes where the receiving party is not relevant (either server process will do).
This way of communication is different from *broadcasting*, where both servers receive the same data value. Broadcasting is not supported in this simulation tool.

In case of two senders, `S0` and `S1`, and one receiver `E` the selection process is the same.
If one of the two servers `S` can communicate with exit `E`, communication between that server and the exit takes place.
If both servers can communicate, a random choice is made. 
Having several senders and several receivers for a single channel is also handled in the same manner.
A random choice is made for the sending process and a random choice is made for the receiving process before each communication. To communicate with several other processes but without non-determinism, unique channels must be used.

Below is the model for the configuration with two servers.

In [ ]:
@process
def Generator(env, c_out):
    for i in range(5):
        yield env.timeout(1)
        x = f"Entity {i} passed through G"
        yield env.execute(c_out.send(x))

@process
def Server(env, c_in, c_out, s):
    while True:
        x = yield env.execute(c_in.receive())
        x = x + f", S{s}"
        yield env.execute(c_out.send(x))

@process
def Exit(env, c_in):
    while True:
        x = yield env.execute(c_in.receive())
        print(x + " and E.")

def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    
    G = Generator(env, a)
    S0 = Server(env, a, b, 0)
    S1 = Server(env, a, b, 1)
    
    E = Exit(env, b)
    
    env.run()
    
model()

## 8.4 Many channels
Multiple channels can be defined at once using list comprehension.
This can be useful in combination with list comprehension for processes.

Below is an example of using list comprehension to quickly create three channels at once for three parallel production processes.

| Figure 8.4: Three parallel production processes |
- 
<img src="figures/8-4.png" width=50%>
<a id='fig:8-4'></a>

The model for this example is shown below. We can re-use the process functions defined in the previous example; only the model needs to be redefined.

In [ ]:
def model():
    N = 3 # the number of parallel processes
    
    env = Environment()
    a = Channel(env)
    b = [Channel(env) for i in range(N)]
    c = Channel(env)
    
    G = Generator(env, a)
    S012 = [Server(env, a, b[i], i) for i in range(N)]
    S345 = [Server(env, b[i], c, i+N) for i in range(N)]
    E = Exit(env, c)
    
    env.run()
    
model()

## 8.5 Monitoring multiple communication events
In some situations it might be necessary for a process to monitor multiple communication events at once. For example, when a process wants to send one entity across either of two channels, which means we need to monitor both communication events to watch if either of them accepts the entity, and we must then execute only one of the communication events (not over both!).

We can achieve this using the `select` function: `Environment.select(communication event 1, communication event 2, ...)`. This function takes multiple communication events as input. When the select function is used, the simulation monitors if any of the communication events can occur, and then executes the first one that can. If multiple communication events are able to occur at the same time, one is *selected* at random (hence the name of the function). The communication events which were not selected are aborted. The end result is the same as if the selected communication event was executed using `Environment.execute(selected communication event)`.

This function accepts both communication events, as well as lists of communication events (lists must be preceded by an asterisk `*`). Examples are:
- Multiple communication events as input: `Environment.select(communication event 1, communication event 2, ...)`
- A list of communication events as input: `Environment.select(*a list of communication events)`
- A combination of the two as input: `Environment.select(*list of communication events 1-3, communication event 4, ...)`

The output when yielding a select function `output = yield Environment.select(...)` depends on what type of communication event was selected; the function returns the received entity as output when a receive event was selected, and it returns `None` as output when a send event was selected.

Below is an example of a provider which is able to provide across two different channels.

In [ ]:
@process
def DualProvider(env, c_out1, c_out2):
    for x in range(10):
        sending1 = c_out1.send(x)
        sending2 = c_out2.send(x)
        yield env.select(sending1, sending2) # we try to send the entity over both channels a and b

@process
def Consumer(env, c_in, i):
    while True:
        x = yield env.execute(c_in.receive())
        print(f"Consumer {i} received entity {x}")
        
    
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    DP = DualProvider(env, a, b)
    C1 = Consumer(env, a, 1)
    C2 = Consumer(env, b, 2)
    env.run()
    
model()

### 8.5.1 Describing the outcomes by using the selected function
In some instances, a process will take a different action depending on which communication event was selected. We can check if a communication event was selected by using the `selected(communication event)` function, which returns `True`  if the communication event has occurred. This is useful when we want to describe different outcomes depending on which event was selected.

The example below shows how we can check if either `sending1` or `sending2` has been selected. The same example also gives an example of using a list of communication events as input for the select function.

In [ ]:
@process
def DualProvider(env, c_out1, c_out2):
    for x in range(10):
        sending1 = c_out1.send(x)
        sending2 = c_out2.send(x)
        communication_events = [sending1, sending2]
        yield env.select(*communication_events)
        if selected(sending1):
            print(f"Provider sent entity {x} across channel a")
        if selected(sending2):
            print(f"Provider sent entity {x} across channel b")
            
@process
def Consumer(env, c_in, i):
    while True:
        x = yield env.execute(c_in.receive())
            
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    DP = DualProvider(env, a, b)
    C1 = Consumer(env, a, 1)
    C2 = Consumer(env, b, 2)
    C3 = Consumer(env, a, 3)
    C4 = Consumer(env, b, 4)
    env.run()
    
model()

### 8.5.2 Guards
Suppose that a process needs to watch different sets of communication events in different scenario's. We could describe every different scenario explicitely, but this would lead to bloated code. Instead, we can add a *guard* to our communication events. A guard can be very useful when a process needs to watch multiple different communication events at the same time, with some communication events only being allowed under certain conditions. 

A guard is a boolean function which denotes under which conditions a communication event is allowed to take place. A sending event with a guard would look as follows:
- `sending = channel.send(entity) if guard_sending else None`
  - with `guard_sending` being a boolean expression denoting if sending is allowed to take place.

We can use guards this way because the `Environment.select(...)` function skips any communication event which has as value `None`. If there are no communication events for which the guard is `True`, then the process continues without any communication.

Below is an example of an the dual provider using guards. In this scenario:
- It only sends across channel1 if `x` is a multiple of 2.
- It only sends across channel2 if `x` is a multiple of 3.

This translates to `guard1 = (x % 2 == 0)` for `sending1` and `guard2 = (x % 3 == 0)` for `sending2`. The result is:
- If both `guard1` and `guard2` are true (e.g. `x=6`), select either `sending1` or `sending2` randomly 
- If either of the guards is true, and the other is false (e.g. `x=3`), select the corresponding communication event.
- If neither of the guards is true (e.g. `x=1`), throw away entity `x`.

In [ ]:
@process
def DualProvider(env, c_out1, c_out2):
    for x in range(1,21):
        guard1 = (x % 2 == 0)  # The guard for sending across channel c_out1: only send when x is a multiple of 2
        guard2 = (x % 3 == 0)  # The guard for sending across channel c_out2: only send when x is a multiple of 3
        
        sending1 = c_out1.send(x) if guard1 else None  # only execute sending1 when guard1 is satisfied
        sending2 = c_out2.send(x) if guard2 else None  # only execute sending2 when guard1 is satisfied
        
        communication_events = [sending1, sending2]
        yield env.select(*communication_events)
        if selected(sending1):
            print(f"Provider sent entity {x} across channel a")
        if selected(sending2):
            print(f"Provider sent entity {x} across channel b")
        if not selected(sending1) and not selected(sending2):
            print(f"Provider throws away entity {x}")

@process
def Consumer(env, c_in, i):
    while True:
        x = yield env.execute(c_in.receive())            
            
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    DP = DualProvider(env, a, b)
    C1 = Consumer(env, a, 1)
    C2 = Consumer(env, b, 2)
    env.run()
    
model()

## 8.6 Summary

- A channel is defined with `channel = Channel(environment)`
- Communication over a channel is done using communication events.
    - A process can send an entity over a channel with communication event `sending = channel.send(entity)`
    - A process can receive over a channel with communication event `receiving = channel.receive()` 
    - Communication events do not start automatically, they need to be executed through `env.execute(communication event)`
- A channel can transmit any type of entity; for example integers, strings, or even a custom data type
    - a Signal without data is named a *synchronization signal* and is done through: `channel.send(None)`
- A channel can have several processes sending and/or receiving.
    - If multiple processes want to send/receive at the same time, then the channel will select a sender and a receiver at random.
- Multiple channels can be defined using list comprehension, which can be useful in combination with list comprehension for processes.
	- An example would be `channels = [Channel(env) for i in range(10)]`
- Monitoring multiple channels 
     - A process can select one of multiple communication events. Only the first communication event which is able to communicate will be executed, the others are aborted. If multiple communication events are able to communicate at the same time, then one is chosen at random. This can be done through:
        - `env.select(communication event 1, communication event 2, ...)`
        - `env.select(*list of communication events)`
        - or with a combination of the two: `env.select(*list of communication events 1-3, communication event 4, ...)`
     - We can check if a communication event was selected by using the `selected(communication event)` function, which returns `True`  if the communication event has occurred. This is useful when we want to describe different outcomes depending on which event was selected.
     - Guards are boolean functions which denotes under which conditions a communication event is allowed to take place. 
        - An example of a send event with a guard would be: `sending = channel.send(entity) if guard_sending else None`
          - with `guard_sending` being a boolean expression denoting if sending is allowed to take place.

## 8.7 Exercises
1. Given is the specification of process `P` and model `M`. Why does the model terminate immediately?



In [ ]:
@process
def P(env, c_in, c_out):
    x = 0
    while True:
        x = yield env.execute(c_in.receive())
        x = x + 1
        print(x)   
        yield env.execute(c_out.send())

def M():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
   
    P1 = P(a,b)
    P2 = P(b,a)

    env.run()

2. Six children have been given the assignment to perform a series of calculations on the numbers $0,1,2,3,\ldots ,9$, namely add 2, multiply by 3, multiply by 2, and add 6 subsequently. They decide to split up the calculations and to operate in parallel. They sit down at a table next to each other. The first child, the reader $R$, reads the numbers $0,1,2,3,\ldots ,9$ one by one to the first calculating child $C_1$. Child $C_1$ adds 2 and tells the result to its right neighbour, child $C_2$.  After telling the result to child $C_2$, child $C_1$ is able to start calculating on the next number the reader $R$ tells him. Children $C_2$, $C_3$, and $C_4$ are analogous to child $C_1$; they each perform a different calculation on a number they hear and tell the result to their right neighbour. At the end of the table the writer $W$ writes every result he hears down on paper. The figure below shows a schematic drawing of the children at the table.


| Figure 8.5: The six children |
- 
<img src="figures/8-5.png" width=75%>
<a id='fig:8-5'></a>

a. Finish the specification for the reading child $R$, that reads the numbers 0 till 9 one by one.

In [ ]:
@process
def R(env, ...):
    i = 0
    while i < 10:
        ...

b. Specify the parameterized process $C_{\textit{add}}$ that represents the children $C_1$ and $C_4$, who perform an addition.

In [ ]:
@process
def C_add(env, ...):
    while True:
        ...

c. Specify the parameterized process $C_{\textit{mul}}$ that represents the children $C_2$ and $C_3$, who perform a multiplication.

In [ ]:
@process
def C_mul(env, ...):
    while True:
        ...

d. Specify the process $W$ representing the writing child. Print each result as output.

In [ ]:
@process
def W(env, a):
    while True:
        ...

e. Make a graphical representation of the model `SixChildren` that is composed of the six children.

f. Specify the model `SixChildren`. Simulate the model.

In [ ]:
def SixChildren():
    env = Environment()
    
    ...

    env.run()
    
SixChildren()

## 8.7 Answers to exercises

<details>
    <summary>[Click for the answer to a]</summary>

    Answer:
    
```python
@process
def R(env, c_out):
    i = 0
    while i < 10:
        yield env.execute(c_out.send(i))
```
</details>
    
<details>
    <summary>[Click for the answer to b]</summary>

    Answer:
    
```python
@process
def C_add(env, c_in, c_out, addition):
    while True:
        x = yield env.execute(c_in.receive())
        x = x + addition
        yield env.execute(c_out.send(x))
```
  
</details>
    
<details>
    <summary>[Click for the answer to c]</summary>

    Answer:
    
```python
@process
def C_mul(env, c_in, c_out, multiplier):
    while True:
        x = yield env.execute(c_in.receive())
        x = x * multiplier
        yield env.execute(c_out.send(x))
```
</details>
    
<details>
    <summary>[Click for the answer to d]</summary>

    Answer:
    
```python
@process
def W(env, c_in):
    while True:
        x = yield env.execute(c_in.receive())
        print(x)
```
</details>

<details>
    <summary>[Click for the answer to e]</summary>

Answer:

| Figure 8.6: The six children |
-
<img src="figures/8-6.png" width=75%>
<a id='fig:8-6'></a>


</details>

<details>
    <summary>[Click for the answer to f]</summary>

    Answer:
    
```python
def SixChildren():
    env = Environment()
    
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    d = Channel(env)
    e = Channel(env)
    
    reader = R(env, a)
    C1 = C_add(env, a, b, 2)
    C2 = C_mul(env, b, c, 3)
    C3 = C_mul(env, c, d, 2)
    C4 = C_add(env, d, e, 6)
    writer = W(env, e)
    
    env.run()
    
SixChildren()
```
</details>
    